In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.optimizers import AdamW
AUTOTUNE = tf.data.AUTOTUNE
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LambdaCallback

In [ ]:
import pandas as pd 
#import cv2 
from PIL import Image 
import tensorflow
from tensorflow import keras
import tensorflow.keras as keras
from tensorflow.keras import layers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D 
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense 
from tensorflow.keras.optimizers import Adam 
#import tensorflow.compat.v1 as tf
import os


In [ ]:
base_dir = '/home/Melspectro_featmaps'
img_size=224
batch_size=64


In [ ]:
# Create a data augmentor 
train_datagen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, 
								zoom_range=0.2, horizontal_flip=True, 
								validation_split=0.2) 

test_datagen = ImageDataGenerator(rescale=1. / 255, 
								validation_split=0.2) 

# Create datasets 
train_datagen = train_datagen.flow_from_directory(base_dir, 
												target_size=( 
													img_size, img_size), 
												subset='training', 
												) 
test_datagen = test_datagen.flow_from_directory(base_dir, 
												target_size=( 
													img_size, img_size), 
												subset='validation', 
												) 

In [ ]:
len (train_datagen)

In [ ]:
len (test_datagen)

In [ ]:
import tqdm
train_datagen.reset()
x_train, y_train= next(train_datagen)
for i in tqdm.tqdm(range(int(train_datagen.n/batch_size)-1)): 
  img, label = next(train_datagen)
  x_train = np.append(x_train, img, axis=0 )
  y_train = np.append(y_train, label, axis=0)
print(x_train.shape, y_train.shape)



In [ ]:
import tqdm
test_datagen.reset()
x_test, y_test = next(test_datagen)
for i in tqdm.tqdm(range(int(test_datagen.n/batch_size)-1)): 
  img, label = next(test_datagen)
  x_test = np.append(x_test, img, axis=0 )
  y_test = np.append(y_test, label, axis=0)
print(x_test.shape, y_test.shape)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i])
plt.show()


In [ ]:
num_classes = 5
input_shape = (224, 224, 3)


In [ ]:
batch_size = 64
num_epochs = 100
validation_split = 0.05
weight_decay = 1e-2
label_smoothing = 0.05

steps_per_epoch = 15000 // 64  # = 234
decay_steps = 234 * 100         # = 23400

#lr_scheduler = tf.keras.optimizers.schedules.CosineDecay(
    #initial_learning_rate=1e-4,
    #decay_steps=decay_steps,
    #alpha=1e-5)  # Final LR fraction of initial LR


In [ ]:
num_train_samples = int(len(x_train) * (1 - validation_split))
num_val_samples = len(x_train) - num_train_samples
x_train, x_val = np.split(x_train, [num_train_samples])
y_train, y_val = np.split(y_train, [num_train_samples])
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

print(x_val.shape, y_val.shape)

early_stopping = tf.keras.callbacks.EarlyStopping(    monitor='val_loss',        # What metric to monitor
    patience=5,                # Stop if no improvement after 5 epochs
    restore_best_weights=True # Roll back to best weights after stopping
)


In [ ]:
def window_partition(x, window_size):
    B, H, W, C = tf.shape(x)[0], x.shape[1], x.shape[2], x.shape[3]
    x = tf.reshape(x, shape=(B, H // window_size, window_size, W // window_size, window_size, C))
    x = tf.transpose(x, perm=(0, 1, 3, 2, 4, 5))
    windows = tf.reshape(x, shape=(-1, window_size, window_size, C))
    return windows


In [ ]:
def window_reverse(windows, window_size, height, width, channels):
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(windows, shape=(-1, patch_num_y, patch_num_x, window_size, window_size, channels))
    x = tf.transpose(x, perm=(0, 1, 3, 2, 4, 5))
    x = tf.reshape(x, shape=(-1, height, width, channels))
    return x


In [ ]:
class DropPath(layers.Layer):
    def __init__(self, drop_prob=None, **kwargs):
        super().__init__(**kwargs)
        self.drop_prob = drop_prob

    def call(self, x):
        if self.drop_prob == 0. or not self.trainable:
            return x
        input_shape = tf.shape(x)
        batch_size = input_shape[0]
        rank = x.shape.rank
        shape = (batch_size,) + (1,) * (rank - 1)
        random_tensor = (1 - self.drop_prob) + tf.random.uniform(shape, dtype=x.dtype)
        path_mask = tf.floor(random_tensor)
        return tf.math.divide(x, 1 - self.drop_prob) * path_mask


In [ ]:
class WindowAttention(layers.Layer):
    def __init__(self, dim, window_size, num_heads, qkv_bias=True, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5
        self.qkv = layers.Dense(dim * 3, use_bias=qkv_bias)
        self.dropout = layers.Dropout(dropout_rate)
        self.proj = layers.Dense(dim)

    def build(self, input_shape):
        num_window_elements = (2 * self.window_size[0] - 1) * (2 * self.window_size[1] - 1)
        self.relative_position_bias_table = self.add_weight(
            shape=(num_window_elements, self.num_heads),
            initializer=tf.initializers.Zeros(),
            trainable=True,
        )
        coords_h = np.arange(self.window_size[0])
        coords_w = np.arange(self.window_size[1])
        coords_matrix = np.meshgrid(coords_h, coords_w, indexing="ij")
        coords = np.stack(coords_matrix)
        coords_flatten = coords.reshape(2, -1)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.transpose([1, 2, 0])
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)

        self.relative_position_index = tf.Variable(
            initial_value=tf.convert_to_tensor(relative_position_index), trainable=False
        )

    def call(self, x, mask=None):
        _, size, channels = x.shape
        head_dim = channels // self.num_heads
        x_qkv = self.qkv(x)
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, 3, self.num_heads, head_dim))
        x_qkv = tf.transpose(x_qkv, perm=(2, 0, 3, 1, 4))
        q, k, v = x_qkv[0], x_qkv[1], x_qkv[2]
        q = q * self.scale
        k = tf.transpose(k, perm=(0, 1, 3, 2))
        attn = q @ k

        num_window_elements = self.window_size[0] * self.window_size[1]
        relative_position_index_flat = tf.reshape(self.relative_position_index, shape=(-1,))
        relative_position_bias = tf.gather(self.relative_position_bias_table, relative_position_index_flat)
        relative_position_bias = tf.reshape(
            relative_position_bias, shape=(num_window_elements, num_window_elements, -1)
        )
        relative_position_bias = tf.transpose(relative_position_bias, perm=(2, 0, 1))
        attn = attn + tf.expand_dims(relative_position_bias, axis=0)

        if mask is not None:
            nW = mask.get_shape()[0]
            mask_float = tf.cast(tf.expand_dims(tf.expand_dims(mask, axis=1), axis=0), tf.float32)
            attn = tf.reshape(attn, shape=(-1, nW, self.num_heads, size, size)) + mask_float
            attn = tf.reshape(attn, shape=(-1, self.num_heads, size, size))
            attn = keras.activations.softmax(attn, axis=-1)
        else:
            attn = keras.activations.softmax(attn, axis=-1)

        attn = self.dropout(attn)
        x_qkv = attn @ v
        x_qkv = tf.transpose(x_qkv, perm=(0, 2, 1, 3))
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, channels))
        x_qkv = self.proj(x_qkv)
        x_qkv = self.dropout(x_qkv)
        return x_qkv


In [ ]:
class SwinTransformer(layers.Layer):
    def __init__(self, dim, num_patch, num_heads, window_size=7, shift_size=0,
                 num_mlp=1024, qkv_bias=True, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.num_patch = num_patch
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.num_mlp = num_mlp

        self.norm1 = layers.LayerNormalization(epsilon=1e-5)
        self.attn = WindowAttention(
            dim, window_size=(self.window_size, self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, dropout_rate=dropout_rate
        )
        self.drop_path = DropPath(dropout_rate)
        self.norm2 = layers.LayerNormalization(epsilon=1e-5)

        self.mlp = keras.Sequential([
            layers.Dense(num_mlp),
            layers.Activation(keras.activations.gelu),
            layers.Dropout(dropout_rate),
            layers.Dense(dim),
            layers.Dropout(dropout_rate),
        ])

        if min(self.num_patch) < self.window_size:
            self.shift_size = 0
            self.window_size = min(self.num_patch)

    def build(self, input_shape):
        if self.shift_size == 0:
            self.attn_mask = None
        else:
            height, width = self.num_patch
            h_slices = (slice(0, -self.window_size), slice(-self.window_size, -self.shift_size), slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size), slice(-self.window_size, -self.shift_size), slice(-self.shift_size, None))
            mask_array = np.zeros((1, height, width, 1))
            count = 0
            for h in h_slices:
                for w in w_slices:
                    mask_array[:, h, w, :] = count
                    count += 1
            mask_windows = window_partition(mask_array, self.window_size)
            mask_windows = tf.reshape(mask_windows, (-1, self.window_size * self.window_size))
            attn_mask = np.expand_dims(mask_windows, axis=1) - np.expand_dims(mask_windows, axis=2)
            attn_mask = np.where(attn_mask != 0, -100.0, 0.0)
            self.attn_mask = tf.constant(attn_mask, dtype=tf.float32)

    def call(self, x):
        height, width = self.num_patch
        _, num_patches_before, channels = x.shape
        x_skip = x
        x = self.norm1(x)
        x = tf.reshape(x, shape=(-1, height, width, channels))
        if self.shift_size > 0:
            shifted_x = tf.roll(x, shift=[-self.shift_size, -self.shift_size], axis=[1, 2])
        else:
            shifted_x = x

        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = tf.reshape(x_windows, shape=(-1, self.window_size * self.window_size, channels))
        attn_windows = self.attn(x_windows, mask=self.attn_mask)
        attn_windows = tf.reshape(attn_windows, shape=(-1, self.window_size, self.window_size, channels))
        shifted_x = window_reverse(attn_windows, self.window_size, height, width, channels)

        if self.shift_size > 0:
            x = tf.roll(shifted_x, shift=[self.shift_size, self.shift_size], axis=[1, 2])
        else:
            x = shifted_x

        x = tf.reshape(x, shape=(-1, height * width, channels))
        x = self.drop_path(x)
        x = x_skip + x
        x_skip = x
        x = self.norm2(x)
        x = self.mlp(x)
        x = self.drop_path(x)
        x = x_skip + x
        return x


In [ ]:
class PatchExtract(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size_x = patch_size[0]
        self.patch_size_y = patch_size[1]

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=(1, self.patch_size_x, self.patch_size_y, 1),
            strides=(1, self.patch_size_x, self.patch_size_y, 1),
            rates=(1, 1, 1, 1),
            padding="VALID",
        )
        patch_dim = patches.shape[-1]
        patch_num = patches.shape[1]
        return tf.reshape(patches, (batch_size, patch_num * patch_num, patch_dim))


In [ ]:
class PatchEmbedding(layers.Layer):
    def __init__(self, num_patches, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patches = num_patches
        self.proj = layers.Dense(embed_dim)
        self.pos_embed = layers.Embedding(input_dim=num_patches, output_dim=embed_dim)

    def call(self, patch):
        pos = tf.range(start=0, limit=self.num_patches, delta=1)
        return self.proj(patch) + self.pos_embed(pos)


In [ ]:
image_dimension=224

In [ ]:
class PatchMerging(tf.keras.layers.Layer):
    def __init__(self, num_patch, embed_dim):
        super().__init__()
        self.num_patch = num_patch
        self.embed_dim = embed_dim
        self.linear_trans = layers.Dense(2 * embed_dim, use_bias=False)

    def call(self, x):
        height, width = self.num_patch
        _, _, C = x.get_shape().as_list()
        x = tf.reshape(x, shape=(-1, height, width, C))
        x0 = x[:, 0::2, 0::2, :]
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = tf.concat((x0, x1, x2, x3), axis=-1)
        x = tf.reshape(x, shape=(-1, (height // 2) * (width // 2), 4 * C))
        return self.linear_trans(x)


In [ ]:
def build_mswmsa_t(input_shape=(224, 224, 3), num_classes=5,
                 patch_size=4, embed_dim=96,
                 depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24],
                 window_size=7, mlp_ratio=4., qkv_bias=True, dropout_rate=0.1):

    input = layers.Input(shape=input_shape)
    x = PatchExtract(patch_size=(patch_size, patch_size))(input)

    num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)
    patch_resolution = (input_shape[0] // patch_size, input_shape[1] // patch_size)

    x = PatchEmbedding(num_patches=num_patches, embed_dim=embed_dim)(x)

    # Stage 1 (depth=2)
    for i in range(2):
        shift_size = 0 if i % 2 == 0 else window_size // 2
        print(f"Stage 1 - {'W-MSA' if shift_size == 0 else 'SW-MSA'}_{i + 1}")
        x = SwinTransformer(
            dim=embed_dim, num_patch=patch_resolution, num_heads=num_heads[0],
            window_size=window_size, shift_size=shift_size,
            num_mlp=int(embed_dim * mlp_ratio), qkv_bias=qkv_bias,
            dropout_rate=dropout_rate
        )(x)

    # Patch Merging 1
    x = PatchMerging(patch_resolution, embed_dim)(x)
    patch_resolution = (patch_resolution[0] // 2, patch_resolution[1] // 2)
    embed_dim *= 2

    # Stage 2 (depth=2)
    for i in range(2):
        shift_size = 0 if i % 2 == 0 else window_size // 2
        print(f"Stage 2 - {'W-MSA' if shift_size == 0 else 'SW-MSA'}_{i + 1}")
        x = SwinTransformer(
            dim=embed_dim, num_patch=patch_resolution, num_heads=num_heads[1],
            window_size=window_size, shift_size=shift_size,
            num_mlp=int(embed_dim * mlp_ratio), qkv_bias=qkv_bias,
            dropout_rate=dropout_rate
        )(x)

    # Patch Merging 2
    x = PatchMerging(patch_resolution, embed_dim)(x)
    patch_resolution = (patch_resolution[0] // 2, patch_resolution[1] // 2)
    embed_dim *= 2

    # Stage 3 (depth=6)
    for i in range(6):
        shift_size = 0 if i % 3 == 0 else window_size // 2
        print(f"Stage 3 - {'W-MSA' if shift_size == 0 else 'SW-MSA'}_{i + 1}")
        x = SwinTransformer(
            dim=embed_dim, num_patch=patch_resolution, num_heads=num_heads[2],
            window_size=window_size, shift_size=shift_size,
            num_mlp=int(embed_dim * mlp_ratio), qkv_bias=qkv_bias,
            dropout_rate=dropout_rate
        )(x)

    # Patch Merging 3
    x = PatchMerging(patch_resolution, embed_dim)(x)
    patch_resolution = (patch_resolution[0] // 2, patch_resolution[1] // 2)
    embed_dim *= 2

    # Stage 4 (depth=2)
    for i in range(2):
        shift_size = 0 if i % 2 == 0 else window_size // 2
        print(f"Stage 4 - {'W-MSA' if shift_size == 0 else 'SW-MSA'}_{i + 1}")
        x = SwinTransformer(
            dim=embed_dim, num_patch=patch_resolution, num_heads=num_heads[3],
            window_size=window_size, shift_size=shift_size,
            num_mlp=int(embed_dim * mlp_ratio), qkv_bias=qkv_bias,
            dropout_rate=dropout_rate
        )(x)


    # Classification Head
    x = layers.LayerNormalization(epsilon=1e-5)(x)
    x = layers.GlobalAveragePooling1D()(x)
    output = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs=input, outputs=output)
    return model


In [ ]:
model = build_mswmsa_t()
model.summary()

In [ ]:
import time
import os
import json
import tensorflow as tf

class ResumeTimeCallback(tf.keras.callbacks.Callback):
    def __init__(self, save_dir="checkpoints"):
        super().__init__()
        self.save_dir = save_dir
        self.total_train_time = 0
        self.total_val_time = 0
        self.epoch_times = []  # Store per-epoch timing

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_train_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_train_time = time.time() - self.epoch_train_start
        self.total_train_time += epoch_train_time

        # Save this epoch's times
        epoch_time_info = {
            "epoch": epoch + 1,
            "train_time_sec": round(epoch_train_time, 2),
            "val_time_sec": round(self.last_val_time, 2) if hasattr(self, 'last_val_time') else 0.0
        }
        self.epoch_times.append(epoch_time_info)

        # Print per-epoch times
        print(f"\n Epoch {epoch+1} - Train Time: {epoch_time_info['train_time_sec']}s | Val Time: {epoch_time_info['val_time_sec']}s")

        # Save epoch timing info to file
        with open(os.path.join(self.save_dir, "epoch_times.json"), "w") as f:
            json.dump(self.epoch_times, f, indent=2)

    def on_test_begin(self, logs=None):
        self.val_start_time = time.time()

    def on_test_end(self, logs=None):
        self.last_val_time = time.time() - self.val_start_time
        self.total_val_time += self.last_val_time

    def on_train_end(self, logs=None):
        print(f"\n Total Training Time: {self.total_train_time:.2f} sec")
        print(f" Total Validation Time: {self.total_val_time:.2f} sec")

In [ ]:
experiment_name = "mswmsa"
checkpoint_dir = f"checkpoints_{experiment_name}"

os.makedirs(checkpoint_dir, exist_ok=True)

# Save/load epoch functions
def save_epoch(epoch):
    with open(os.path.join(checkpoint_dir, "epoch.json"), "w") as f:
        json.dump({"epoch": epoch}, f)

def load_epoch():
    try:
        with open(os.path.join(checkpoint_dir, "epoch.json")) as f:
            return json.load(f)["epoch"]
    except:
        return 0

In [ ]:
latest_epoch = load_epoch()
if latest_epoch > 0:
    model.load_weights(os.path.join(checkpoint_dir, "model.weights.h5"))
    print(f"Resumed from epoch {latest_epoch}")
else:
    print("Starting from scratch")


In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, "model.weights.h5"),
    save_weights_only=True,
    save_best_only=False,
    save_freq='epoch'
)

In [ ]:
resume_time_cb = ResumeTimeCallback(save_dir=checkpoint_dir)

In [ ]:
def augment(x):
    x = tf.image.random_crop(x, size=(image_dimension, image_dimension, 3))
    x = tf.image.random_flip_left_right(x)
    return x

dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .map(lambda x, y: (augment(x), y))
    .batch(batch_size=batch_size)
    #.map(lambda x, y: (patch_extract(x), y))
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset_val = (
    tf.data.Dataset.from_tensor_slices((x_val, y_val))
    .batch(batch_size=batch_size)
    #.map(lambda x, y: (patch_extract(x), y))
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset_test = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(batch_size=batch_size)
    #.map(lambda x, y: (patch_extract(x), y))
    .prefetch(tf.data.experimental.AUTOTUNE)
)


In [ ]:

model.compile(
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing),
    optimizer=tf.keras.optimizers.experimental.AdamW(
        learning_rate=1e-4, weight_decay=1e-2
    ),
    metrics=[
        keras.metrics.CategoricalAccuracy(name="accuracy"),
        keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),
    ],
)



In [ ]:
print("Training steps per epoch:", len(dataset))

In [ ]:
len(dataset_val)

In [ ]:
model.save("my_mswmsa_model.keras")

In [ ]:
history = model.fit(
    dataset,
    validation_data=dataset_val, 
    epochs=num_epochs,
    steps_per_epoch = len(dataset),
    validation_steps = len(dataset_val),
    initial_epoch=latest_epoch,
    callbacks=[
        checkpoint_cb,
        LambdaCallback(on_epoch_end=lambda epoch, logs: save_epoch(epoch + 1)),
        resume_time_cb,
        early_stopping
    ]
)

In [ ]:
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train and Validation Losses Over Epochs", fontsize=14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
loss, accuracy, top_5_accuracy = model.evaluate(dataset_test)
print(f"Test loss: {round(loss, 2)}")
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")


In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')

plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')

plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
pred = model.predict(dataset_test)

In [ ]:
predicted=[]
labels=[]
for im, label in dataset_test :
    predicted.append(model(im))
    labels.append(label.numpy())

In [ ]:
pred= np.concatenate([np.argmax(predicted[:-1],axis=-1).flatten(),np.argmax(predicted[-1],axis=-1).flatten()])
lab= np.concatenate([np.argmax(labels[:-1],axis=-1).flatten(),np.argmax(labels[-1],axis=-1).flatten()])

In [ ]:
class_name=['classA','classB','classC','classD','classE']

In [ ]:
plt.figure(figsize=(12,12))
for   images,labels in  dataset_test.take(10):
  for i   in range(16):
   ax=plt.subplot(4,4,i+1)

   plt.imshow(images[i])
   plt.title("True Label :"+ class_name[tf.argmax(labels[i],axis=0).numpy()] + "\n" + "Predicted Label  : " + class_name[tf.argmax(model(tf.expand_dims(images[i],axis=0)),axis= -1).numpy()[0]])
   plt.axis("off")

In [ ]:
import seaborn as sns
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix
cm=confusion_matrix(lab,pred)
print(cm)
plt.figure(figsize=(8,8))
sns.heatmap(cm,annot=True)
plt.title("Confusion Matrix")
plt.ylabel('Actual')
plt.xlabel("Predicted")